# 1) Import Package

In [1]:
import pandas
import csv
import re
import time
import datetime
import numpy
from io import StringIO
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.cryptocurrencies import CryptoCurrencies

# 2) Input API

In [2]:
API_KEYS = "SXFMNCRU1PXNVHST"
#LIMIT --> 5 API Requests per Minutes
#LIMIT --> 500 API Requests per Day
ts_e = TimeSeries(key = API_KEYS, output_format="csv") 

# 3) Scrape Raw Data 

In [4]:
#timing is slicer
# Scrape Raw data and write into temp file
def scrape_ext_temp_write(stockname, timing):
    data_e , metadata_e = ts_e.get_intraday_extended(symbol=stockname, interval="1min", slice=timing)
    temp_filename = (f"TEMP_{stockname}_{timing}.csv")
    with open(temp_filename, 'w') as f:
        writer = csv.writer(f)
        # write the data
        writer.writerow(data_e)

# 4) Convert Raw Data to proper format

In [5]:
def scrape_ext_temp_processor(stockname, timing):
    temp_filename = (f"TEMP_{stockname}_{timing}.csv")
    temp1 = pandas.read_csv(temp_filename)
    temp2 = temp1.columns.values.tolist()
    data_e_use = pandas.DataFrame(columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    temp3 = temp2[1:]
    q = 0
    
#    for i in range (len(temp3)): 
    for i in range (1,len(temp3)):  
        temp4 = temp3[i].split(",")

        temp5 = list()
        for k in range(q,(q+5)) :
            temp5.append(temp4[k][2:-1])

        temp5.append(temp4[(q+5)][2:-2])

        a_series = pandas.Series(temp5, index=['time', 'open', 'high', 'low', 'close', 'volume'])
        a_series["time"] = datetime.datetime.strptime(a_series["time"], '%Y-%m-%d %H:%M:%S')
        a_series["open"] = pandas.to_numeric(a_series["open"], downcast="float")
        a_series["high"] = pandas.to_numeric(a_series["high"], downcast="float")
        a_series["low"] = pandas.to_numeric(a_series["low"], downcast="float")
        a_series["close"] = pandas.to_numeric(a_series["close"], downcast="float")
        a_series["volume"] = pandas.to_numeric(a_series["volume"], downcast="float")
        a_series["Stock_Name"] = stockname
        data_e_use = data_e_use.append(a_series, ignore_index=True)
        
    Timing_final_filename = (f"Timing FINAL_{stockname}_{timing}.csv")
    data_e_use.to_csv(Timing_final_filename)

# 5) Call 3 & 4 by looping

In [6]:
def scrape_ext(stockname):
    Full_Slice = ["year1month1", "year1month2", "year1month3", "year1month4", "year1month5", "year1month6", "year1month7", "year1month8", "year1month9", "year1month10", "year1month11", "year1month12", "year2month1", "year2month2", "year2month3", "year2month4", "year2month5", "year2month6", "year2month7", "year2month8", "year2month9", "year2month10", "year2month11", "year2month12"]
    for each_timing in Full_Slice:
        error_token = 0
        try:
            scrape_ext_temp_write(stockname, each_timing)
        except:
            print(f"during {stockname}_{each_timing} writing temp file exception raise, error count = {error_token+1} time")
            if error_token < 3:
                print(f"{stockname}_{each_timing} from error = {error_token+1} if (token < 3) was called")
                error_token = error_token+1
                time.sleep(60)
                pass
            else:
                print(f"during {stockname}_{each_timing} writing temp error else (token >= 3) was called")
                continue
                
        error_token = 0
        try:  
            scrape_ext_temp_processor(stockname, each_timing)
        except:
            print(f"during {stockname}_{each_timing} processing file exception raise, error count = {error_token+1} time")
            if error_token < 3:
                print(f"{stockname}_{each_timing} from error = {error_token+1} if (token < 3) was called")
                error_token = error_token+1
                time.sleep(60)
                pass
            else:
                print(f"during {stockname}_{each_timing} processing temp error else (token >= 3) was called")
                continue

In [7]:
scrape_ext("BA")

In [ ]:
scrape_ext("RTX")

In [ ]:
scrape_ext("GD")

In [ ]:
scrape_ext("HII")

In [ ]:
scrape_ext("NOC")

In [ ]:
scrape_ext("TXT")

In [ ]:
scrape_ext("LDOS")

In [ ]:
scrape_ext("BWXT")

In [ ]:
scrape_ext("LHX")

In [ ]:
scrape_ext("CACI")

In [ ]:
scrape_ext("MANT")